In [1]:
import os

import mlflow

In [2]:

%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer\\research'

In [3]:
NUMEXPR_MAX_THREADS = 20

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path


In [7]:
@dataclass(frozen=True)
class CredentialsConfig:
    MLFLOW_TRACKING_URI: str
    MLFLOW_TRACKING_USERNAME: str
    MLFLOW_TRACKING_PASSWORD: str


In [8]:
from mlops_NLP_Text_Summarization.constants import *
from mlops_NLP_Text_Summarization.utils.common import read_yaml, create_directories, save_json

In [9]:
class Credentials:
    def __init__(
        self,
        secrets_filepath = SECRETS_FILE_PATH):
        

        self.secret= read_yaml(secrets_filepath)
    
    def get_mlflow_tracking_credentials(self) -> CredentialsConfig:
        secret = self.secret
        
        model_evaluation_config = CredentialsConfig(
            MLFLOW_TRACKING_URI=self.secret.MLFLOW_TRACKING_URI,
            MLFLOW_TRACKING_USERNAME=self.secret.MLFLOW_TRACKING_USERNAME,
            MLFLOW_TRACKING_PASSWORD = self.secret.MLFLOW_TRACKING_PASSWORD                  
           
        )
        return {
            "MLFLOW_TRACKING_URI": model_evaluation_config.MLFLOW_TRACKING_URI,
            "MLFLOW_TRACKING_USERNAME": model_evaluation_config.MLFLOW_TRACKING_USERNAME,
            "MLFLOW_TRACKING_PASSWORD": model_evaluation_config.MLFLOW_TRACKING_PASSWORD
        }
    

In [10]:
credentials = Credentials()

[2023-08-09 13:44:26,480: INFO: common: yaml file: secrets.yaml loaded successfully]


In [11]:
mlflow_credentials = credentials.get_mlflow_tracking_credentials()

In [12]:
os.environ["MLFLOW_TRACKING_URI"] = mlflow_credentials["MLFLOW_TRACKING_URI"]

In [13]:
os.environ["MLFLOW_TRACKING_USERNAME"] = mlflow_credentials["MLFLOW_TRACKING_USERNAME"]

In [14]:
os.environ["MLFLOW_TRACKING_PASSWORD"] = mlflow_credentials["MLFLOW_TRACKING_PASSWORD"]

In [15]:
PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION="python"

In [16]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    params: dict
    tokenizer_path: Path
    metric_file_name: Path
    mlflow_uri: str
    experiment_name: str
    model_path_packed: Path

In [17]:
from mlops_NLP_Text_Summarization.constants import *
from mlops_NLP_Text_Summarization.utils.common import read_yaml, create_directories, save_json


In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        secrets_filepath = SECRETS_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.secrets = read_yaml(secrets_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.TrainingArguments
        secrets = self.secrets

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            params = params,            
            metric_file_name = config.metric_file_name,
            mlflow_uri = secrets.MLFLOW_TRACKING_URI,
            experiment_name=config.experiment_name,
            model_path_packed= config.model_path_packed,
            
        )

        return model_evaluation_config

In [23]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm
from urllib.parse import urlparse
import json
import joblib
import yaml

In [24]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        

        #name of experiment
        if mlflow.active_run():
            mlflow.end_run()


        mlflow.set_experiment(self.config.experiment_name)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        def get_experiment_id(name):
            exp = mlflow.get_experiment_by_name(name)
            if exp is None:
                exp_id = mlflow.create_experiment(name)
                return exp_id
            return exp.experiment_id

        
        def get_last_run_id(exp_name):
            exp = get_experiment_id(exp_name)
            client = mlflow.MlflowClient()
            runs = client.search_runs(experiment_ids=exp)
            if len(runs) == 0:
                return None
            last_run = runs[0]
            return last_run.info.run_id

            run_id = get_last_run_id(self.config.experiment_name)
            print(run_id)
            os.environ[sys.argv[1]] = run_id
            print(f'Saving {run_id} to environment variable {sys.argv[1]}')

            run_id = os.environ[sys.argv[1]]
      

# Select a name for the model to be registered


        
        with mlflow.start_run(run_name=self.config.experiment_name):

            run_id = get_last_run_id(self.config.experiment_name)

            subpath = "text_summarization"

            model_name = "samsum_pegasus_model"

            # build the run URI
            run_uri = f'runs:/{run_id}/{subpath}'

            # register the model
            


            rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    
            rouge_metric = load_metric('rouge')

            score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
                )
            
            rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
            

            print(rouge_dict)
            #df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
            
            
            save_json(path=Path(self.config.metric_file_name), data=rouge_dict)
            mlflow.log_params(self.config.params)
            mlflow.log_metric('rouge1', rouge_dict['rouge1'])
            mlflow.log_metric('rouge2', rouge_dict['rouge2'])
            mlflow.log_metric('rougeL', rouge_dict['rougeL'])
            mlflow.log_metric('rougeLsum', rouge_dict['rougeLsum'])

            model_version = mlflow.register_model(run_uri, model_name)


        if tracking_url_type_store != "file":
            mlflow.pytorch.log_model(model_pegasus, artifact_path= self.config.model_path, registered_model_name="samsum_pegasus_model_reg_name")
        else:
            print( "logging model to local file")


            

    #def log_into_mlflow(self):

#keras_model=model,
#                       artifact_path=MODELS_DIR,
#                       registered_model_name="Ultimate Mario Model")
#        
        

        #with open(self.config.metric_file_name, "r") as f:
        #    data = f.readlines()
#
        #for line in data:
        #    line = line.strip()
        #    values = line.split(",")
        #    if len(values) == 2:
        #        metric_name, metric_value = values
        #        mlflow.log_metric(metric_name, float(metric_value))
        #        
        #mlflow.log_artifact(self.config.metric_file_name)
#
        #with open("params.yaml", "r") as f:
        #    params = yaml.safe_load(f)
        #    
        #mlflow.end_run()
        #with mlflow.start_run():
        #    mlflow.log_params(params)
        #    self.evaluate()
            



In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()    
except Exception as e:
    raise e
    

[2023-08-09 15:02:42,963: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-09 15:02:42,967: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-09 15:02:42,969: INFO: common: yaml file: secrets.yaml loaded successfully]
[2023-08-09 15:02:42,970: INFO: common: created directory at: artifacts]
[2023-08-09 15:02:42,971: INFO: common: created directory at: artifacts/model_evaluation]


2023/08/09 15:02:53 INFO mlflow.tracking.fluent: Experiment with name 'pegasus-samsum-model_test' does not exist. Creating a new experiment.
100%|██████████| 5/5 [02:16<00:00, 27.23s/it]

[2023-08-09 15:05:12,025: INFO: rouge_scorer: Using default tokenizer.]


{'rouge1': 0.02185158469332997, 'rouge2': 0.0, 'rougeL': 0.021873432881271068, 'rougeLsum': 0.021867135284620094}
[2023-08-09 15:05:12,244: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'samsum_pegasus_model' already exists. Creating a new version of this model...
2023/08/09 15:05:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: samsum_pegasus_model, version 8
Created version '8' of model 'samsum_pegasus_model'.


[2023-08-09 15:17:47,157: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /Nkifor/mlops-nlp-text-summarizer.mlflow/api/2.0/mlflow-artifacts/artifacts/7c3315d8806244699f2dddb8e4262f96/bbaf84c16a4b4f5084201bbdef8dfc71/artifacts/artifacts/model_training/pegasus-samsum-model/data/model.pth]
[2023-08-09 15:30:08,229: WARNING: connectionpool: Retrying (Retry(total=3, connect=5, read=3, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /Nkifor/mlops-nlp-text-summarizer.mlflow/api/2.0/mlflow-artifacts/artifacts/7c3315d8806244699f2dddb8e4262f96/bbaf84c16a4b4f5084201bbdef8dfc71/artifacts/artifacts/model_training/pegasus-samsum-model/data/model.pth]
[2023-08-09 15:42:33,616: WARNING: connectionpool: Retrying (Ret

KeyboardInterrupt: 